## IMPORT

In [1]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

from os import getenv, listdir
from os.path import isfile, join

from dotenv import load_dotenv
import datetime
from datetime import date

## DB 정보 로딩

In [2]:
load_dotenv(dotenv_path = 'insert_data/db_info.env')

DB_HOST = getenv('DB_HOST', None)
DB_PORT = getenv('DB_PORT', 5432)
DB_USER = getenv('DB_USER', None)
DB_PASS = getenv('DB_PASS', None)
DB_NAME = getenv('DB_NAME', None)

conn_string = f"host='{DB_HOST}' dbname='{DB_NAME}' user='{DB_USER}' password='{DB_PASS}' port={DB_PORT}"
conn = pg.connect(conn_string)

## DATA LOAD

In [3]:
def select_all(table_name: str):
    return eval(f"psql.read_sql('SELECT * FROM {table_name}', conn)")

In [4]:
person_data = select_all(table_name="person") # 환자에 대한 정보
death_data = select_all(table_name="death") # 환자의 사망 정보 
visit_occurrence_data = select_all(table_name="visit_occurrence") # 방문에 대한 정보
condition_occurrence_data = select_all(table_name="condition_occurrence") # 병명에 대한 정보
drug_exposure_data = select_all(table_name="drug_exposure") # 의약품 처방 정보

## 과제 문서에 언급된 column만 선정하여 데이터를 재가공

In [5]:
persons = person_data[["person_id", "gender_concept_id", "ethnicity_concept_id", "year_of_birth"]]
persons.head(2)

,person_id,gender_concept_id,ethnicity_concept_id,year_of_birth
0,402435,8532,0,1997
1,1022983,8507,0,1950


In [6]:
death = death_data[["person_id", "death_date"]]
death.head(2)

,person_id,death_date
0,1691806,2015-06-02
1,99181,2018-11-04


In [7]:
visit_occurrences = visit_occurrence_data[["person_id", "visit_occurrence_id", "visit_concept_id", "visit_end_date", "visit_start_date"]]
visit_occurrences.head(2)

,person_id,visit_occurrence_id,visit_concept_id,visit_end_date,visit_start_date
0,116496,36112943,9202,1962-04-13,1962-04-13
1,116496,36112944,9202,1962-04-24,1962-04-24


In [8]:
condition_occurrences = condition_occurrence_data[["person_id", "visit_occurrence_id", "condition_concept_id", "condition_source_value"]]
condition_occurrences.head(2)

,person_id,visit_occurrence_id,condition_concept_id,condition_source_value
0,116496,36112954,0,162864005
1,116496,36112952,0,840544004


In [9]:
drug_exposures = drug_exposure_data[["person_id", "visit_occurrence_id", "drug_concept_id", "drug_source_value"]]
drug_exposures.head(2)

,person_id,visit_occurrence_id,drug_concept_id,drug_source_value
0,26922,99499216,19073183,308182
1,2955,9251642,40231925,1049221


### person 데이터와 visit_occurrence 데이터의 새로운 컬럼 생성
- person: 나이 (year_of_birth)
- visit_occurrence: 내원일수 (visit_end_date - visit_start_date)

In [10]:
today = date.today().strftime("%Y")
persons["age"] = int(today) - persons["year_of_birth"] + 1 # 한국은 만나이가 아니므로 1을 더해줌

# year_of_birth 컬럼 제거
if "year_of_birth" in list(persons.columns):
    persons = persons.drop(["year_of_birth"], axis=1)

<ipython-input-10-8f18ad4e47dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persons["age"] = int(today) - persons["year_of_birth"] + 1 # 한국은 만나이가 아니므로 1을 더해줌


In [11]:
persons.head(2)

,person_id,gender_concept_id,ethnicity_concept_id,age
0,402435,8532,0,24
1,1022983,8507,0,71


In [12]:
visit_occurrences["visit"] = pd.to_datetime(visit_occurrences['visit_end_date']) \
                                - pd.to_datetime(visit_occurrences['visit_start_date'])

# visit_end_date, visit_start_date 컬럼 제거
for c in visit_occurrences.columns:
    if c == "visit_end_date" or c == "visit_start_date":
        visit_occurrences = visit_occurrences.drop([c], axis=1)

<ipython-input-12-77196e6bb3e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visit_occurrences["visit"] = pd.to_datetime(visit_occurrences['visit_end_date']) \


In [13]:
visit_occurrences.head(3)

,person_id,visit_occurrence_id,visit_concept_id,visit
0,116496,36112943,9202,0 days
1,116496,36112944,9202,0 days
2,116496,36112954,9202,0 days


## JOIN DATA
- person_id와 visit_occurrence_id로 join하여 데이터를 하나로 만든다.

In [14]:
print(len(persons), len(death), len(visit_occurrence_data), len(condition_occurrence_data), len(drug_exposure_data))

1000 152 41810 12167 46579


#### person과 death를 조인하면 사망한 사람의 정보를 알 수 있음

In [15]:
new_data = pd.merge(persons, death, how='left', on='person_id')
new_data.tail(2)

,person_id,gender_concept_id,ethnicity_concept_id,age,death_date
998,2565313,8532,0,56,NaN
999,277792,8507,0,67,NaN


#### person+death+visit_occurrence

In [16]:
new_data = pd.merge(new_data, visit_occurrences, on='person_id')
new_data.tail(2)

,person_id,gender_concept_id,ethnicity_concept_id,age,death_date,visit_occurrence_id,visit_concept_id,visit
41808,277792,8507,0,67,NaN,67292323,9202,0 days
41809,277792,8507,0,67,NaN,67292311,9202,0 days


#### person+death+visit_occurrence+condition_occurrences

In [17]:
new_data = pd.merge(new_data, condition_occurrences, on=['person_id', 'visit_occurrence_id'])
new_data.tail(2)

,person_id,gender_concept_id,ethnicity_concept_id,age,death_date,visit_occurrence_id,visit_concept_id,visit,condition_concept_id,condition_source_value
12165,277792,8507,0,67,NaN,67292311,9202,0 days,0,840544004
12166,277792,8507,0,67,NaN,67292311,9202,0 days,37311061,840539006


#### person+death+visit_occurrence+condition_occurrences

In [18]:
new_data = pd.merge(new_data, drug_exposures, on=['person_id', 'visit_occurrence_id'])
new_data.tail(2)

,person_id,gender_concept_id,ethnicity_concept_id,age,death_date,visit_occurrence_id,visit_concept_id,visit,condition_concept_id,condition_source_value,drug_concept_id,drug_source_value
8204,2565313,8532,0,56,NaN,41635384,9202,0 days,4294548,75498004,1713671,562251
8205,277792,8507,0,67,NaN,67292308,9202,0 days,260139,10509002,1127433,313782


### COPY DATA

In [29]:
data_copy = new_data.copy()
data_copy.tail(2)

,person_id,gender_concept_id,ethnicity_concept_id,age,death_date,visit_occurrence_id,visit_concept_id,visit,condition_concept_id,condition_source_value,drug_concept_id,drug_source_value
8204,2565313,8532,0,56,NaN,41635384,9202,0 days,4294548,75498004,1713671,562251
8205,277792,8507,0,67,NaN,67292308,9202,0 days,260139,10509002,1127433,313782


### NaN value 체크

In [30]:
print(data_copy.isnull().sum())

person_id                    0
gender_concept_id            0
ethnicity_concept_id         0
age                          0
death_date                5436
visit_occurrence_id          0
visit_concept_id             0
visit                        0
condition_concept_id         0
condition_source_value       0
drug_concept_id              0
drug_source_value            0
dtype: int64


## 문제풀이

사망률과 연관성이 높은 변수 탐색

- person: 성별, 인종, 나이
- visit_occurrence: 입원, 외래, 응급, 내원일수
- condition_occurrence: 진단
- drug_exposure: 의약품

**사용할 연관성 척도: 두 변수 사이의 상관관계의 정도를 나타내는 상관계수를 이용 한다.**

### DUMMIES

- 사망한 사람인 경우 0으로 사망하지 않은 사람의 경우 1로 변경

In [31]:
data_copy['death_date'] = data_copy['death_date'].apply(lambda x:1 if pd.isna(x) else 0)
data_copy.tail(2)

,person_id,gender_concept_id,ethnicity_concept_id,age,death_date,visit_occurrence_id,visit_concept_id,visit,condition_concept_id,condition_source_value,drug_concept_id,drug_source_value
8204,2565313,8532,0,56,1,41635384,9202,0 days,4294548,75498004,1713671,562251
8205,277792,8507,0,67,1,67292308,9202,0 days,260139,10509002,1127433,313782


### 1. 나이와 사망률의 상관계수
- 나이는 연속형 변수로도 볼 수도 있어서 더미 변환하지 않음

In [32]:
data_copy['age'].value_counts()

66     307
59     298
70     287
71     206
72     198
      ... 
100     13
98      11
3       11
1       10
112      4
Name: age, Length: 112, dtype: int64

In [57]:
age_corr_data = pd.concat([data_copy['death_date'], data_copy['age']], axis=1)
age_corr_data.corr()

,death_date,age
death_date,1.00000,-0.46894
age,-0.46894,1.00000


### 2. 인종과 사망률의 상관계수
- 한 종만 있어서 비교대상에서 제외 (인종이 사망률에 영향을 주는지 확인하려면 여러 종의 데이터가 있어야 한다)

In [34]:
data_copy['ethnicity_concept_id'].value_counts()

0    8206
Name: ethnicity_concept_id, dtype: int64

### 3. 성별과 사망률의 상관계수
- 8507: -0.092428
- 8532: -0.092428

In [35]:
data_copy['gender_concept_id'].value_counts()

8507    4888
8532    3318
Name: gender_concept_id, dtype: int64

In [59]:
# 성별 더미 변환
gender_dummies = pd.get_dummies(data_copy['gender_concept_id'])
gender_corr_data = pd.concat([data_copy['death_date'], gender_dummies], axis=1)
gender_corr_data.corr().head(1)

,death_date,8507,8532
death_date,1.0,-0.092428,0.092428


In [ ]:
# 상관계수 평균

### 4. 입원/외래/응급과 사망률과의 상관계수
- 입원: -0.129634
- 외래: 0.111581
- 응급: 0.017198

In [38]:
data_copy['visit_concept_id'].value_counts()

9202    4548
9201    2709
9203     949
Name: visit_concept_id, dtype: int64

In [62]:
# 더미 변환
v_concept_dummies = pd.get_dummies(data_copy['visit_concept_id'])
v_concept_corr_data = pd.concat([data_copy['death_date'], v_concept_dummies], axis=1)
v_concept_corr_data.tail(2)

,death_date,9201,9202,9203
8204,1,0,1,0
8205,1,0,1,0


In [63]:
v_concept_corr_data.corr().head(1)

,death_date,9201,9202,9203
death_date,1.0,-0.129634,0.111581,0.017198


### 5. 내원일수와 사망률의 상관계수
- 연속형 변수로 바라보고 더미 변환하지 않음
- 내원일수: -0.05147

In [67]:
data_copy['visit'].value_counts()

0 days      5249
10 days      406
9 days       382
1 days       335
13 days      320
11 days      206
12 days      202
8 days       197
18 days      195
14 days      149
16 days      130
15 days      109
5 days        74
6 days        70
19 days       38
3 days        31
4 days        26
22 days       25
17 days       24
7 days        15
20 days       15
345 days       3
370 days       2
62 days        2
24 days        1
Name: visit, dtype: int64

In [70]:
v_visit_corr_data = pd.concat([data_copy['death_date'], data_copy['visit']], axis=1)
v_visit_corr_data = v_visit_corr_data.astype(int)
v_visit_corr_data.tail(2)

,death_date,visit
8204,1,0
8205,1,0


In [71]:
v_visit_corr_data.corr().head(1)

,death_date,visit
death_date,1.0,-0.05147


### 6. 진단과 사망률과의 상관계수
- condition_concept_id: 0.007726 (상관계수 평균)
- condition_source_value: 0.006643 (상관계수 평균)

In [59]:
condition_occurrences['condition_concept_id'].value_counts()

0           1332
40481087    1051
37311061     710
437663       649
4112343      569
            ... 
43530685       2
4282096        1
198809         1
436676         1
196456         1
Name: condition_concept_id, Length: 151, dtype: int64

In [60]:
condition_occurrences['condition_source_value'].value_counts()

444814009    1051
840544004     737
840539006     710
386661006     649
195662009     569
             ... 
235919008       1
36923009        1
65275009        1
47505003        1
48333001        1
Name: condition_source_value, Length: 160, dtype: int64

In [72]:
# 더미 변환
c_concept_dummies = pd.get_dummies(data_copy['condition_concept_id'])
c_concept_corr_data = pd.concat([data_copy['death_date'], c_concept_dummies], axis=1)
c_concept_corr_data.tail(2)

,death_date,0,28060,31967,75036,77074,78272,80502,81151,192279,...,40480160,40481087,40486433,43530652,43530656,43530685,44782746,45768439,45769905,45770830
8204,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8205,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
c_concept_corr = c_concept_corr_data.corr().head(1)
c_concept_corr

,death_date,0,28060,31967,75036,77074,78272,80502,81151,192279,...,40480160,40481087,40486433,43530652,43530656,43530685,44782746,45768439,45769905,45770830
death_date,1.0,-0.007561,0.050908,0.007881,0.009853,0.007881,0.021344,0.01382,0.057838,-0.016539,...,0.002342,0.062329,0.012031,0.012066,0.007579,0.013651,0.010574,-0.019795,-0.01331,0.015521


In [78]:
c_concept_corr.drop('death_date')
c_concept_corr.mean(axis=1)

death_date    0.007726
dtype: float64

In [80]:
# 더미 변환
c_source_dummies = pd.get_dummies(data_copy['condition_source_value'])
c_source_corr_data = pd.concat([data_copy['death_date'], c_source_dummies], axis=1)
c_source_corr_data.tail(2)

,death_date,5602001,7200002,10509002,15777000,16114001,19169002,22298006,26929004,30832001,...,770349000,840539006,840544004,1501000119109,1551000119108,90781000119102,97331000119101,124171000119105,132281000119108,368581000119106
8204,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8205,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
c_source_corr = c_source_corr_data.corr().head(1)
c_source_corr.drop('death_date')
c_source_corr.mean(axis=1)

death_date    0.006643
dtype: float64

### 7. 의약품과 사망률의 상관계수
- drug_concept_id: 0.005741 (상관계수 평균)
- drug_source_value: 0.005741 (상관계수 평균)

In [82]:
drug_exposures['drug_concept_id'].value_counts()

40213154    8015
19078106    3669
19009384    2825
40224172    2814
19127663    2729
            ... 
35605380       1
43012036       1
46275507       1
19078924       1
19079524       1
Name: drug_concept_id, Length: 164, dtype: int64

In [83]:
drug_exposures['drug_source_value'].value_counts()

140        8015
310798     3669
106892     2825
999967     2814
746030     2729
           ... 
1363309       1
1657981       1
727762        1
1234995       1
1659263       1
Name: drug_source_value, Length: 164, dtype: int64

In [84]:
# 더미 변환
d_concept_dummies = pd.get_dummies(data_copy['drug_concept_id'])
d_concept_corr_data = pd.concat([data_copy['death_date'], d_concept_dummies], axis=1)
d_concept_corr_data.tail(2)

,death_date,723042,782047,836659,920300,964261,1101556,1114379,1115171,1127433,...,40232448,40236824,40241958,40243436,42629347,42707627,46275123,46275444,46275507,46275916
8204,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8205,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [85]:
d_concept_corr = d_concept_corr_data.corr().head(1)
d_concept_corr.mean(axis=1)

death_date    0.005741
dtype: float64

In [86]:
# 더미 변환
d_source_dummies = pd.get_dummies(data_copy['drug_source_value'])
d_source_corr_data = pd.concat([data_copy['death_date'], d_source_dummies], axis=1)
d_source_corr_data.tail(2)

,death_date,3,10,20,21,33,43,49,52,113,...,1790099,1804799,1807513,1808217,1860154,1860480,2001499,2047241,2103182,2123111
8204,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8205,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
d_source_corr = d_source_corr_data.corr().head(1)
d_source_corr.mean(axis=1)

death_date    0.005741
dtype: float64

## 결론

- 연관성이 높은 순서
1. person-나이 (상관계수 절대값: 0.46894)
2. person-입원 (상관계수 절대값 0.129634)
3. visit_occurrence-외래 (상관계수 절대값 0.111581)
4. person-성별 (상관계수 절대값 0.092428)
5. visit_occurrence-내원일수 (상관계수 절대값 0.05147)
6. visit_occurrence-응급 (상관계수 절대값: 0.017198)
7. condition_occurrence-진단
    - condition_concept_id: 0.007726 (상관계수 평균)
    - condition_source_value: 0.006643 (상관계수 평균)
8. drug_exposure-의약품
    - drug_concept_id: 0.005741 (상관계수 평균)
    - drug_source_value: 0.005741 (상관계수 평균)
9. 판단에서 제외: 인종 (현재 데이터에 인종이 한 종류만 있어서 비교대상에서 제외)